In [2]:
import requests
import pandas as pd
from datetime import datetime, timedelta



StatementMeta(, 10b44e08-f3d9-4856-af80-35c44349bcf1, 4, Finished, Available, Finished, False)

### Chargement Initiale 

In [3]:
def build_data_url(date):
    return "https://raw.githubusercontent.com/gsoulat/data-training-fabric/main/eolienne/" + date + "_wind_power_data.csv"


# initial_date = "20240615"
# initial_url = build_data_url(initial_date)
# print(f"Lien du fichier initiale : {initial_url}")

StatementMeta(, 10b44e08-f3d9-4856-af80-35c44349bcf1, 5, Finished, Available, Finished, False)

In [4]:
# df = pd.read_csv(initial_url)
# df["date"] = pd.to_datetime(df["date"])
# 
# print("Nombre de lignes : ", len(df))
# df.head(5)

StatementMeta(, 10b44e08-f3d9-4856-af80-35c44349bcf1, 6, Finished, Available, Finished, False)

In [5]:
# Chemin vers la table Bronze 

# les ids sont dispo dans l'url de la page du lakehouse. Example :
# https://app.fabric.microsoft.com/groups/79daad76-9b51-4f79-b1a3-5049702b5055/lakehouses/c9815a16-8f35-44be-8356-61d5566edb18
workspace_id = "79daad76-9b51-4f79-b1a3-5049702b5055" 
lakehouse_id = "c9815a16-8f35-44be-8356-61d5566edb18" 

bronze_table_path = f"abfss://{workspace_id}@onelake.dfs.fabric.microsoft.com/{lakehouse_id}/Tables/dbo/wind_power"


StatementMeta(, 10b44e08-f3d9-4856-af80-35c44349bcf1, 7, Finished, Available, Finished, False)

In [6]:
# df_spark = spark.createDataFrame(df)
# df_spark.write.format("delta").mode("overwrite").save(bronze_table_path)

# alternative :
# df_spark.write.format("delta").mode("overwrite").saveAsTable("wind_power")

StatementMeta(, 10b44e08-f3d9-4856-af80-35c44349bcf1, 8, Finished, Available, Finished, False)

### Verification des données

In [11]:
# df_read = spark.read.format("delta").load(bronze_table_path).toPandas()

# alternative : 
# df_read = spark.read.format("delta").table("wind_power").toPandas()

# print("Nombre de lignes : ", len(df_read))
# print("colonnes : ", df_read.columns)
# df_read.head(5)


StatementMeta(, 10b44e08-f3d9-4856-af80-35c44349bcf1, 13, Finished, Available, Finished, False)

Nombre de lignes :  432
colonnes :  Index(['production_id', 'date', 'time', 'turbine_name', 'capacity',
       'location_name', 'latitude', 'longitude', 'region', 'status',
       'responsible_department', 'wind_speed', 'wind_direction',
       'energy_produced'],
      dtype='object')


,production_id,date,time,turbine_name,capacity,location_name,latitude,longitude,region,status,responsible_department,wind_speed,wind_direction,energy_produced
0,6643,2024-06-16,09-00-00,Turbine A,2200,Location 1,34.0522,-118.2437,Region A,Online,Operations,10.14153,W,967.44241
1,6644,2024-06-16,09-00-00,Turbine B,2000,Location 2,36.7783,-119.4179,Region B,Online,Operations,9.52359,NE,1887.41288
2,6645,2024-06-16,09-00-00,Turbine C,2500,Location 3,40.7128,-74.0060,Region C,Online,Operations,5.85461,W,979.52389
3,6646,2024-06-16,09-10-00,Turbine A,2200,Location 1,34.0522,-118.2437,Region A,Online,Operations,24.54081,S,905.83065
4,6647,2024-06-16,09-10-00,Turbine B,2000,Location 2,36.7783,-119.4179,Region B,Online,Operations,13.86345,SW,642.42290


### Import de la date suivante

In [8]:
max_date =pd.to_datetime(spark.read.format("delta").load(bronze_table_path).toPandas()["date"], format="%Y%m%d").max()

next_date = (max_date + timedelta(days=1)).strftime("%Y%m%d")

print(max_date, next_date)

StatementMeta(, 10b44e08-f3d9-4856-af80-35c44349bcf1, 10, Finished, Available, Finished, False)

2024-06-15 00:00:00 20240616


In [10]:
next_day_url = build_data_url(next_date)

df = pd.read_csv(next_day_url)
df["date"] = pd.to_datetime(df["date"])
df_spark = spark.createDataFrame(df)
df_spark.write.format("delta").mode("overwrite").save(bronze_table_path)

StatementMeta(, 10b44e08-f3d9-4856-af80-35c44349bcf1, 12, Finished, Available, Finished, False)